# FIT5196 Task 2 in Assessment 1
#### Student Name: Vyom Chauhan
#### Student ID: 30830192

Date: 13/09/2020

Version: 1.0

Environment: Python 2.7.11 and Jupyter notebook

 </font>

This task is to extract unigrams, bigrams and make a vocab and count vector for the given twitter data present in the excel files. Firstly, we combine the data from different sheets into a single dataframe and then extract only the english tweets from the dataframe. Simultaneously we create a dictionary with date as key and the english tweets as value. Next we import the stopwords file and make a set of stopwords. Next we tokenize the data and store it into a dictionary. For extracting unigrams we firstly remove the stopwords and then stem the remaining tokens using porter stemmer. Next we extract the top 100 unigrams using the FreqDist function and create a string to store the unigrams in a file according to the given format. Next to extract bigrams we use the ngrams function setting n=2 on the tokenized text and extract the top 100 bigrams using the FreqDist function. Next we write it to the bigram file using a string according to the format given. To make a vocabulary of the tweets, firstly we remove the stopwords and then append the tokens into a list to remove the context dependent and rare words. Next, we stem the remaining tokens and remove tokens with length less than 3. Netx to create top 200 bigrams using the pmi measure we use the stopwords removed list and flatten it. We add the bigrams list to the vocab and sort it. Next we write it to the vocab file along with its index. To make the count vectors we use the mwetokenizer instead of the regex tokenizer given so that we can keep the bigrams in the text. Next we perform the operations of removing stopwords, rare words etc. and pass the dictionary as a list to the count vectorizer. Next we create a dictionary of the token index and its count and write it to the countvec file in the format specified. We notice that the vocab has around 11000 words which also consist of the top 200 bigrams. The rare, context dependent and independent words have been removed from the vocab.

In [ ]:
# importing the required libraries
import pandas as pd
import xlrd
from langid import classify
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
from nltk.collocations import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder
from nltk.util import ngrams
from itertools import chain
from nltk.tokenize import MWETokenizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# variable to store data in the form of a dictionary
twitter_dict = {}
# reading the twitter data
twitter_data = pd.ExcelFile("30830192.xlsx")
# creating the column names
new_header = ['text', 'id', 'created_at']
# for storing data stored in different sheets
for sheet in twitter_data.sheet_names:
    # parsing the sheets
    twitter_df = twitter_data.parse(sheet)
    # dropping columns having all null values
    twitter_df = twitter_df.dropna(how='all', axis = 1)
    # dropping rows with all null values
    twitter_df = twitter_df.dropna(how='all', axis = 0)
    # renaming columns
    twitter_df.columns = new_header
    # ignoring first row if it has column names
    if (twitter_df.columns == twitter_df.iloc[0])[1]:
        twitter_df = twitter_df[1:]
    # filtering english tweets
    twitter_df = twitter_df[list(map(lambda x : classify(str(x))[0]=='en', twitter_df['text']))]
    # storing a sheet with date as key and text as value
    twitter_dict[sheet] = ' '.join(str(x) for x in twitter_df.text.values)

In [ ]:
# reading stopwords file
stop = open("stopwords_en.txt", "r")
# storing stopwords as a list
stopwords_list = stop.read().split()
# creating a set of stopwords
stopwords_set = set(stopwords_list)

In [ ]:
# initializing tokenizer
tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
# tokenization
for key in twitter_dict.keys():
    # converting all text to lower case
    twitter_dict[key] = twitter_dict[key].lower()
    # converting text string to list by using tokenizer
    twitter_dict[key] = tokenizer.tokenize(twitter_dict[key])

In [ ]:
# copying contents of twitter_dict o uni_dict
uni_dict = twitter_dict.copy()
# porter stemmer object
stem = PorterStemmer()
# string to be written in file
uni_text = ''
# making the unigrams
for key in uni_dict.keys():
    # removing stopwords
    uni_dict[key] = [w for w in uni_dict[key] if w not in stopwords_set]
    # stemming the tokens
    uni_dict[key] = [stem.stem(w) for w in uni_dict[key]]
    # storing only the top 100 frequent unigrams
    uni_dict[key] = FreqDist(w for w in uni_dict[key]).most_common(100)
    # appending items to string
    uni_text = uni_text + str(key) + ':' + str(uni_dict[key]) + '\n'

In [ ]:
# creating text file for writing
file = open("30830192_100uni.txt", "w")
# writing to text file
file.write(uni_text) 
# closing file object
file.close()

In [ ]:
# copying contents of twitter_dict to bi_dict
bi_dict = twitter_dict.copy()
# string to be written in file
bi_text = ''
# making the bigrams
for key in bi_dict.keys():
    # extracting n-grams where n=2
    bigrams = ngrams(bi_dict[key], n=2)
    # storing only the top 100 frequent bigrams
    bi_dict[key] = FreqDist(bigrams).most_common(100)
    # appending items to string
    bi_text = bi_text + str(key) + ':' + str(bi_dict[key]) + '\n'

In [ ]:
# creating text file for writing
file = open("30830192_100bi.txt", "w") 
# writing to text file
file.write(bi_text) 
# closing file object
file.close()

In [ ]:
# copying contents of twitter_dict to another dictionary
words_dict = twitter_dict.copy()
# stores stopwords removed unigrams
uni_list = []
# removing stpwords
for key in words_dict.keys():
    words_dict[key] = [w for w in words_dict[key] if w not in stopwords_set]
#     words_dict[key] = [stem.stem(w) for w in words_dict[key]]
#     words_dict[key] = [w for w in words_dict[key] if len(w) > 2]
    uni_list.append(words_dict[key])

# stores frequency distribution for words
# did word appear on a particular date or not
# even if it appears multiple times on a date, freq is still 1
# freq increases only if it appears on another date
freq_dict = FreqDist(list(chain.from_iterable(set(x) for x in uni_list)))

In [ ]:
# dictionary for context dependent words i.e words with freq > 60
freq_context = {k:v for (k,v) in freq_dict.items() if v > 60}
# dictionary for rare words i.e words with freq < 5
freq_rare = {k:v for (k,v) in freq_dict.items() if v < 5}

# cleaning the dictionary further
for key in words_dict.keys():
    # removing the context dependent words
    words_dict[key] = [w for w in words_dict[key] if w not in freq_context.keys()]
    # removing rare words
    words_dict[key] = [w for w in words_dict[key] if w not in freq_rare.keys()]
    # stemming words 
    words_dict[key] = [stem.stem(w) for w in words_dict[key]]
    # removing words with length < 3
    words_dict[key] = [w for w in words_dict[key] if len(w)>2]

In [ ]:
# list to store vocab
vocab = []
# adding words in dictionary to vocab 
for (key, value) in (words_dict.items()):
         for item in value:
            # removing duplicates
            if item not in vocab:
                vocab.append(item) 

In [ ]:
# storing the stopwords removed dictionary in a list and flattening the list
flat_list = [item for sublist in uni_list for item in sublist]
# generating top 200 bigrams using pmi measure
b_m = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(flat_list)
bigrams = finder.nbest(b_m.pmi, 200)
# storing the bigrams into a list and joining them using _
bigram_list = []
for item in bigrams:
    # joining the bigrams with _
    bigram_list.append(item[0] + "_" + item[1])
# appending bigrams to the vocab and sorting list
vocab += bigram_list
vocab = sorted(vocab)
# string for writing to file
vocab_txt = ''
# final vocab with index of words
for item in vocab:
    vocab_txt += item + ':' + str(vocab.index(item)) + '\n'

In [ ]:
# creating text file for writing
file = open("30830192_vocab.txt", "w")
# writing to text file
file.write(vocab_txt) 
# closing file object
file.close()

In [ ]:
# initializing the mwetokenizer
tokenizer = MWETokenizer()
# passing the top 200 bigrams to the tokenizer
mwetokenizer = MWETokenizer(bigrams)
# makes sure that the top 200 bigrams passed remain in the dictionary
mwe_dict = dict((key, mwetokenizer.tokenize(text)) for key, text in twitter_dict.items())
# processing the mwetokenized dictionary
for key in mwe_dict.keys():
    # removing stopwords
    mwe_dict[key] = [w for w in mwe_dict[key] if w not in stopwords_set]
    # removing context dependent words
    mwe_dict[key] = [w for w in mwe_dict[key] if w not in freq_context.keys()]
    # removing rare words
    mwe_dict[key] = [w for w in mwe_dict[key] if w not in freq_rare.keys()]
    # stemming the words
    mwe_dict[key] = [stem.stem(w) for w in mwe_dict[key]]
    # removing words with length < 3
    mwe_dict[key] = [w for w in mwe_dict[key] if len(w)>2]

In [ ]:
# making a list to pass to transform function
text_list = []
# appeding all tokens of a date to one item of a list
for (key, tokens) in mwe_dict.items():
    a = ' '.join(tokens)
    text_list.append(a)

In [ ]:
# initializing the count vectorizer
vectorizer = CountVectorizer()
# passing the list to the count vector function
count_vector = vectorizer.fit_transform(text_list)

In [ ]:
# making a nested dictionary having word and frequency for a date
count_dict = {}
# to traverse the count_vector array
c = 0
# extracting words from the vectorizer
vocab2 = vectorizer.get_feature_names()

# making a nested dictionary
for key in mwe_dict.keys(): 
    count_d[key] = {'':''}
    # showing the frequency of a word or bigram only if it is in the vocab defined earlier
    for (word, count) in zip(vocab2, count_vector.toarray()[c]):
        # if count = 0 don't add the word to dictionary
        if word in vocab and count > 0:
            count_d[key].update({vocab.index(word):count})
    count_d[key].pop('', None)
    c += 1

In [ ]:
# writing the count_vector to file in the format specified
with open('30830192_countVec.txt','w') as file:
    for key, value in count_dict.items():
        f.write(key + ',')
        f.write(",".join(["{}:{}".format(word, count) for word, count in value.items()]))
        f.write('\n')